<a href="https://colab.research.google.com/github/hkrob/DoorOpenDetectionTFlite/blob/main/doorStateTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://developers.google.com/mediapipe/solutions/customization/image_classifier

https://colab.research.google.com/notebooks/snippets/drive.ipynb#scrollTo=TbRbQAYqPTci

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker --quiet

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

door.zip looks like this inside:
door
-closed
--training.jpg (hundreds)
-open
--training.jpg (hundreds)

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/temp/door.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()


In [ ]:
ls -la /tmp/door

In [ ]:
image_path = '/tmp/door'
image_path = os.path.join(os.path.dirname(image_path), 'door')

In [ ]:
print(image_path)
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

In [ ]:
with open('labels.txt', 'w') as file: # open the file in write mode
    for i, element in enumerate(labels): # loop through the array with index
        file.write(f'{i} {element}\n') # write each element with line number and newline


In [ ]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

In [ ]:
spec = image_classifier.SupportedModels.MOBILENET_V2
hparams = image_classifier.HParams(export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [ ]:
# fine tune hparams
hparams = image_classifier.HParams(epochs=10)
hparams = image_classifier.HParams(num_gpus=1)


In [ ]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

In [ ]:
hparams=image_classifier.HParams(epochs=15, export_dir="exported_model_2")
options = image_classifier.ImageClassifierOptions(supported_model=spec,  hparams=hparams)
model_2 = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

In [ ]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

In [ ]:
model.export_model()

In [ ]:
from mediapipe_model_maker import quantization

In [ ]:
quantization_config = quantization.QuantizationConfig.for_int8(train_data)

In [ ]:
model.export_model(model_name="model_quant.tflite", quantization_config=quantization_config)

In [ ]:
ls -lh exported_model

In [ ]:
!cp -r exported_model/* /content/drive/MyDrive/temp

In [ ]:
!ls exported_model
files.download('exported_model/*')